# Metalanguage sets

The lambda notebook metalanguage supports sets and set operations in various ways, largely implemented in the `lamb.meta.sets` module. The set theory here is standard naive set theory, but in the concept of a typed metalanguage. A set in this system should be thought of as containing values of metalanguage expressions, i.e. type-theoretic objects. As an initial example, here is a little set at type `{e}`:

In [ ]:
%te {x_e, y_e, f_<e,e>(x_e), _c1}

## Overview

There are two main ways of building a basic set in the metalanguage.

* **`lamb.meta.sets.ListedSet`** allows constructing finite sets by listing elements, where elements can be arbitrary typed expressions. Invoke in the metalanguage using pythonic `{x,y,...}` notation as above.
* **`lamb.meta.sets.ConditionSet`** allows describing a set by giving a boolean condition on a variable. Invoke using metalanguage binding operator notation, `Set x: ...`.

It is also possible to construct `MetaTerm` sets of concrete elements, though not directly via the metalanguage parser. These are equivalent to `ListedSet`s of concrete elements, and will be produced internally in certain cases (for example, when simplifying expressions that involve quantifying over finite set domains). This document won't deal directly with them, but they should be interchangeable with `ListedSet`s of equivalent meta-elements.

On top of these, various standard operations and relations are supported:

* Set membership, written with the operator `<<`.
* Operations: union, intersection, and difference; written with operators `|`, `&`, and `-` respectively.
* Relations: equality, and (proper) sub/supersethood; written with operators `<=>`, `<=`, `<`, `>=`, `>`.

**Set types** are written with a circumfix type constructor `{X}` for inner type `X`. So for example, `{<e,t>}` is the type of sets of functions from `e` to `t`. While type variables are allowed, genuinely polymorphic sets are not supported, i.e. `X` must resolve to a consistent simple type. One caveat on this: when an empty set is constructed without further guidance, its type is treated as unknown (see below in the `ListedSet` documentation).

`ListedSet`s and `ConditionSet`s are not distinguished at all in the type system.

**General simplification and execution notes**: simplifying set expressions is fairly non-trivial, and the metalanguage simplification code is generally only complete when dealing with "concrete" sets that consist of `MetaTerm`s; this is discussed in detail below. There are two simplify options that are directly relevance; these will try to eliminate set operations, relations, and descriptions to the extent possible. The option `eliminate_sets_all` tries to eliminate everything, and `eliminate_sets` (entailed by the former) tries to eliminate everything but membership statements with listed sets of cardinality >1. This may appear quite ad-hoc at first glance, and it is certainly heuristic, but this is because the general case of $\in$ expressions convert to long, unreadable, and slow-to-execute disjunctions of equalities.

Set code supports compilation/execution fully when dealing with finite domains. Currently, expressions that require quantification do not support compilation for non-finite domains at all (even for cases where this might be in principle possible).

## Set construction and membership testing

This section goes through the main ways of constructing sets, as well as the relation that essentially determines the meta-semantics for sets, namely set membership.

(As usual, it is also possible to have atomic terms with a set type, but these are straightforward.)

### Construction by listing elements (`ListedSet`)

To construct a set with some arbitrary elements, you can use essentially the python set notation to list them, which builds an instance of `meta.sets.ListedSet`:

In [ ]:
%te {True, False}

Sets of sets (etc) are supported by the metalanguage parser:

In [ ]:
%te {{True, False}, {True}, {False}}

In [ ]:
%te {x_e, _c1, _c2}

In [ ]:
%te {P_<e,t>(_c1), Q_<e,t>(_c1)}

The empty set can be written with `{}`. (*note*: this has a different meaning in python code, where `{}` gives you an empty `dict`!) As noted above, without further guidance, the empty set when instantiate this way is handled via a polymorphic type, which allows it to resolve as needed for any concrete set type it might interact with.

In [ ]:
%te {}

In [ ]:
%te {x_e, _c1, _c2} & {}

A cautionary note on semantics for listed sets: sets described by list with elements that contain free terms are surprisingly complicated objects. This is because we can't determine identity without resolving those free terms. As an example, consider whether the two sets `{_c1, x_e}` and `{_c1}` are equivalent. From a quick glance applying standard naive set theory, the answer may appear to be no. However, the answer is a solid *maybe*. In particular, if `x_e` resolves to `_c1`, then the left set is `{_c1, _c1}`, which is equivalent to `{_c1}`. However, if `x` resolves to something else, then they aren't equivalent. We can concretely test this out using equivalence (discussed in detail below):

In [ ]:
x = %te {x_e, _c1} <=> {_c1}
x

In [ ]:
# now test things out by actually doing the substitions described above
# see "simplifcation notes" below for more on step 1 of this first derivation
x.under_assignment({'x': '_c1'}).simplify_all().derivation

In [ ]:
x.under_assignment({'x': '_c2'}).simplify_all().derivation

This scenario is about the simplest case for `ListedSet` semantics with free terms; things get even more complicated with multiple free terms, and complex terms containing free terms.

In general: if a listed set doesn't involve free terms, then everything will go as you expect from standard naive set theory, and simplification of expressions will work. More generally, any set that is concrete in the following sense works completely with all operations and relations:

* Call a an expression **concrete** if it is a `MetaTerm`, or is constructed only from listed sets and tuples of concrete expressions. `{_c1, _c2}` is a concrete set, but `{_c1, x_e}` is not.

**Simplification notes**: there isn't much to simplifying listed sets, with one comment about implementation. As you'd expect from set theory, sets don't care about multiple membership, although it is syntactically valid. That is, a listed set with multiple members is equivalent to the same listed set with duplicates dropped.

The implementation note is that `ListedSet` objects do not enforce non-duplication on construction, but only on simplification. That is, syntactically an object like the following is a perfectly valid `ListedSet` object; it will simplify as expected according to this semantics:

In [ ]:
meta.sets.ListedSet([te("_c1"), te("_c1")])

In [ ]:
meta.sets.ListedSet([te("_c1"), te("_c1")]).simplify()

Unless you are intentionally constructing sets that have syntactically repeated elements, it's rare to see an instance like this in action; it can mainly happen during derivations in certain cases (illustrated by the assignment example above). The reason is that the metalanguage parser for the special case of set construction automatically normalizes, as does the standard factory function `sset` (see below); and whenever one of these sets shows up as a simplification step, it is guaranteed to be resolved.

Also, autonormalization/simplification for listed sets does lead to a normalized display order for elements. This is cosmetic, since of course order doesn't matter for set identity.

In [ ]:
meta.sets.ListedSet([te("x_e"), te("_c2"), te("_c1")]).simplify()

**Set construction in python code**: If you write code that builds `ListedSet` objects, it is recommended to use one of several convenience factories:

`meta.sets.sset` creates and normalizes (in one step, with no derivational history) a `ListedSet` object. It takes an optional parameter `typ` that is the content type of the set.

`meta.sets.emptyset` creates an empty set at an optionally specified type given by parameter `settype`. If this typ is a set type, it is interpreted as the target type (otherwise, as the content type). 

In [ ]:
meta.sets.sset([3,2,3,1])

In [ ]:
meta.sets.emptyset(settype=tp("{n}"))

Also, given a python set container of TypedExprs (or strings parsable to TypedExprs) the function `meta.core.from_python` conveniently produces a `ListedSet` object:

In [ ]:
meta.core.from_python_container({'_c1', te('x_e')})

Python caveat: you can't write sets of sets in python, because the default `set` type is not hashable. Instead, use the `frozenset` type.

### Construction by condition (`ConditionSet`)

Set construction by condition has the same syntax as that of binding operators more generally; boolean constraints on a bound variable determine the contents of the set. This syntax leads to an object of class `meta.sets.ConditionSet`, which is a `BindingOp` subclass.

In [ ]:
%te Set x_e: P_<e,t>(x) & Q_<e,t>(x)

In [ ]:
%te Set x_e: False

Simplification of condition sets, for the most part, is entirely about simplification of their condition. There is one special case which is simplified by conversion to a `ListedSet`, namely, the `False` condition giving the empty set:

In [ ]:
%te simplify Set x_e: False

**Simplification / execution notes**. `ConditionSet`s have two special case simplification checks to be aware of, as these may be useful targets for other simplification code.

* A set of the form `{Set x : False}` will simplify to the `ListedSet` version of the empty set.
* A set that has a single clause in its condition that characterizes its value either ve $\in$ or via an equality will simplify to the exact set the condition characterizes. E.g. `{Set x_e : x <=> _c1}` will simplify to `{_c1}`. This is obviously not a general purpose version of this idea, so it shouldn't be relied on, but as noted above, it is a good target.

In [ ]:
%te simplify {Set x_e : x <=> _c1}
_llast.derivation

In [ ]:
%te simplify {Set x_e : x << {_c1, _c2}}
_llast.derivation

Finally, it is very important to be aware that the general case of `ConditionSet` *does not have* a meta-meta-language counterpart. This is exactly because it is allowed to model non-finiteness, and this situation places some limits on what can be done with these sets.

* When simplifying: there are various cases that just can't be checked without a finite domain restriction (see the documentation on "Domain elements and `MetaTerm`s"). E.g. equivalence checking for arbitrary non-identical `ConditionSet`s. This is to some degree up to boolean simplification, which of course is a non-trivial problem that the lambda notebook doesn't really attempt to solve by itself.
* When compiling/executing: `ConditionSet`s in general won't compile without a finite domain, and an exception will be raised. (The one exception for this is the empty set in `ConditionSet` form.) Where possible, operators that interact with condition sets will attempt to skip this check. For example, `SetContains` (see below) can use the condition directly. Certain operators have special handling for the identities, including `{Set x: True}`. But, working around the possibility of non-finiteness isn't possible in the general case, and compilation doesn't try. Rather, it is recommended to run a static `simplify_all()` pass before executing set expressions that may involve `{Set x: True}`; static inference can deal with these in a general way. See further examples in [#Further-notes-on-metalanguage-sets](#Further-notes-on-metalanguage-sets).
* With a finite domain restriction, executing `ConditionSet`s will work. But be aware that the compiled code must iterate over the whole domain!

### Set membership tests (`SetContains`)

Set identity is fundamentally determined by *membership*: two sets are the same if they have the same members. Therefore, understanding how set membership itself works is crucial towards understanding the metalanguage implementation of sets.

The `<<` operator both in the metalanguage and in python is used to build a set membership expression (of class `meta.sets.SetContains`). This operator takes a set-typed expression on the right, an element on the left with the content type, and produces an expression of type `t`.

Membership checks for concrete elements vs concrete sets are guaranteed to simplify (and execute).

In [ ]:
%te _c1 << {_c1, _c2}

In [ ]:
%te simplify _c1 << {_c1, _c2}
_llast.derivation

Non-concrete simplification with listed sets is quite a bit more complicated, and isn't generally guaranteed. Various special cases will work.

In [ ]:
# singleton set membership checks are just equalities
%te simplify _c1 << {x_e}
_llast.derivation

In [ ]:
# positive membership is easy (this will work for a non-concrete element as well)
%te simplify _c1 << {_c1, x_e}
_llast.derivation

In [ ]:
# nothing is ever in the empty set
%te simplify x_e << {}
_llast.derivation

In [ ]:
# it's guaranteed that _c1 and _c3 are irrelevant
%te simplify _c2 << {_c1, x_e, y_e, _c3}
_llast.derivation

However, many related cases cannot be further simplified. The following example is not resolvable without knowing what `x` is, so simplification by default does nothing:

In [ ]:
%te simplify x_e << {_c1, _c2}

Expressions like this can be "simplified" in certain ways; see below under "simplification notes" for further discussion of non-default simplifications strategies.

Sets of sets (etc) are supported:

In [ ]:
%te simplify {_c1} << {{_c1, _c2}, {_c1}}

Membership checks for `ConditionSet`s are mechanically much more simple, though perhaps not conceptually. Given a condition set, you can straightforwardly check the element against the set's condition. This case always simplifies; the implementation works by constructing a function from the set condition, and doing reduction.

In [ ]:
%te simplify _c1 << (Set x_e : P_<e,t>(x))
_llast.derivation

Both trivial condition cases are handled:

In [ ]:
%te simplify _c1 << (Set x_e : False)

In [ ]:
%te simplify _c1 << (Set x_e : True)

**Simplification notes**. `ConditionSet`s are uniform and straighforward, but as we have already seen, `ListedSet`s are not. Any listed set membership statement can in principle be converted to a disjunction of equality statements. This is generally not what you want to do, as it tends to complicate, rather than simplify, expressions (both in terms of human readability, and consequent evaluation). Therefore, this is only automatically done for the special case of singleton sets. 

With the `eliminate_sets_all` option provided to simplification, this conversion will be forced. This is the only set elimination operation that is *not* triggered by regular `eliminate_sets`! (The "all" option entails `eliminate_sets`.)

In [ ]:
# special case
%te simplify _c2 << {x_e}

In [ ]:
e = %te _c2 << {y_e, x_e}
e.simplify_all(eliminate_sets_all=True).derivation

## Set operations

Here are some examples of union (`|`), intersection (`&`), and difference (`-`). These operations will always simplify for combinations of concrete sets. Combining `ConditionSet`s with `ConditionSet`s via operations also generally works. However, once you move outside these two scenarios, things can become very complicated and are a mess of special cases.

In [ ]:
%te simplify {_c1, _c2} - ({_c2} | {_c3, _c4})
_llast.derivation

In [ ]:
%te simplify (Set x: Q(x)) - (Set x: Q(x))
_llast.derivation

In [ ]:
%te simplify (Set x_e : P_<e,t>(x)) & (Set x_e : Q_<e,t>(x) & P_<e,t>(x))
_llast.derivation

In [ ]:
%te simplify (Set x_e: P_<e,t>(x)) - {_c1, _c2}

**simplification notes**. The `eliminate_sets` option will force certain optional simplification steps that may lead to more complex formulas. In general, this option instructs simplification to try to eliminate all set operations to the extent possible, reducing them to boolean expressions. (Of course, if the expression itself has a set type, not all set descriptions can be eliminated.) Here's an example that works out reasonably:

In [ ]:
s = %te (Set x_e: P_<e,t>(x)) - {_c1, _c2}
s.simplify_all(eliminate_sets=True).derivation.trace()

In [ ]:
# the membership check can be eliminated as well, using `eliminate_sets_all`
s.simplify_all(eliminate_sets_all=True).derivation.trace()

Set elimination strategies are automatically applied (locally) when combining `ConditionSet`s with `ConditionSet`s even without this option, as the simplification system can always find a resulting single `ConditionSet` for these cases. Here's the previous case in a slightly different form:

In [ ]:
%te simplify (Set x_e: P_<e,t>(x)) - (Set x_e: x << {_c1, _c2})

**Execution notes**: While simplification, as seen above, can go through various conversion-to-boolean-expression steps, executing compiled set expressions does not do this (it is never needed, and would generally be slower).

## Set relations

The set membership relation (`SetContains` / `<<`) was dealt with above. this section covers the relations between sets. The metalanguage implements equivalence as well as standard subset and superset relations.

### Equivalence (`SetEquivalence`)

Equivalence is worth singling out, because it is quite a bit trickier than it might first seem. The metalanguage implements the standard idea that two sets are equivalent iff they have the same members. However, testing this in all but the concrete cases becomes challenging.

* Metalanguage class: `meta.sets.SetEquivalence`
* Notation: In parsed metalanguage expressions, like other forms of equality/equivalence, you can use any of `<=>`, `==`, and `%` for this. *However*, be aware that for technical reasons, on the python side, `==` does not give equivalence! The `%` operator does work, and all `TypedExpr`s have an `equivalent_to` member function that will build equivalence expressions.

First, for concrete sets, everything will work as expected.

In [ ]:
%te simplify {{_c1}, {_c2}} <=> {{_c2}, {_c1}}
_llast.derivation

For listed sets with free terms, things become complicated very fast, for reasons we have already seen when discussing the basics of listed sets as well as set membership earlier. As another exmaple, consider the sets `{_c1, x_e}` and `{_c2, y_e}` Are these equivalent? The answer is maybe. In this case there is one way that they could be equivalent, if `y` resolves to `_c1` and `x` resolves to `_c2`. Therefore, there is no default simplification for this case. These expressions can be force-"simplified" by providing the `eliminate_sets` or `eliminate_sets_all` options, but the result is often not particularly simpler.

In [ ]:
s = %te {_c1, x_e} <=> {_c2, y_e}
s.simplify_all(eliminate_sets=True).derivation

In [ ]:
s.simplify_all(eliminate_sets_all=True).derivation

A few special cases of note. Equivalence with empty sets will always simplify, for obvious reasons:

In [ ]:
%te simplify {} <=> {}

In [ ]:
%te simplify {x_e, y_e, _c1} <=> {}
_llast.derivation

And similar to set membership, the case of two singletons always simplifies to an equality expression on the inner type:

In [ ]:
%te simplify {x_e} <=> {_c1}

Certain other more involved special cases for listed sets may simplify.

Equivalence checks for condition sets can generally be simplified to boolean quantified expressions, and will be by default. However, condition sets are not assumed to be finite, and so to do this check in the general case requires potentially non-finite quantification over the relevant domain. Without finiteness, the lambda notebook will not try to simplify these expressions further. (Of course, with domain finiteness, these expressions can be fully simplified or executed with no free terms).

Here is a concrete case of this:

In [ ]:
%te simplify (Set x_e: P_<e,t>(x)) <=> (Set x_e: Q_<e,t>(x))
_llast.derivation

And finally, mixing listed and condition sets is messy, but simplification can be forced with `eliminate_sets`. Unfortunately, the current implementation via simplification does not guarantee finite safety, though this could be done in principle. Here's an example:

In [ ]:
s = %te (Set x_e: P_<e,t>(x)) <=> {x_e, _c1}
s.simplify_all(eliminate_sets_all=True)

**simplification notes**. As seen above, for non-concrete cases simplification is a mix of heuristics that may or may not apply by default. The option `eliminate_sets` guarantees that a set equivalence expression and its component sets will be converted to a boolean expression (potentially quantified), but as you might expect, the result is not necessarily simpler from a human perspective.

**execution notes**. All finite-safe set expressions with no free terms are guaranteed to be executable, with one (current) exception: the mixed listed/condition case noted above for simplification is not handled for execution either.

### Subset and superset relations

The metalanguage provides four further relations:

* `meta.sets.SetSubset`, with operator `<=`, and `meta.sets.SetProperSubset`, operator `<`
* `meta.sets.SetSupset`, with operator `>=`, and `meta.sets.SetProperSupset`, operator `>`

Like equivalence, these take arguments of (matched) set types, and produce a type `t` expression. Here are a few examples of expression construction.

In [ ]:
%te {_c1, _c2} < {_c1, _c2, _c3}

In [ ]:
%te (Set x_e : P_<e,t>(x)) >= (Set x_e : Q_<e,t>(x))

In [ ]:
%te (Set x_e : P_<e,t>(x)) <= {_c1}

**simplification/execution notes**: All of these relations both simplify and execute by means of simpler expressions, and so inherit all the simplification and execution caveats from those. Here are the four implementations.

* `A <= B`: `(A <=> (A & B)`
* `A < B`: `A <= B & ~(A <=> B)`
* `A >= B`: `B <= A`
* `A > B`: `B < A`

As you can see, these in particular rely on intersection, which is reasonably straightforward, and equivalence, which is not. Because of this, simplification and execution (assuming no free terms) do work for all concrete cases and all condition set vs condition set cases, though simplification is not particularly optimized.

Some of the complexity can be seen by looking at simplify derivations for even simple concrete cases:

In [ ]:
%te simplify {_c1, _c2} < {_c1, _c2, _c3}
_llast.derivation

In [ ]:
%te simplify {_c1, _c2, _c4} < {_c1, _c2, _c3}
_llast.derivation

Here are some condition set and mixed examples:

In [ ]:
%te simplify (Set x_e : P_<e,t>(x)) >= (Set x_e : Q_<e,t>(x))
_llast.derivation

In [ ]:
%te simplify ((Set x_e : P_<e,t>(x)) - (Set x_e : Q_<e,t>(x))) >= (Set x_e : Q_<e,t>(x))
_llast.derivation

In [ ]:
s = %te (Set x_e : P_<e,t>(x)) <= {_c1, _c2}
s.simplify_all(eliminate_sets=True).derivation

## Further notes on metalanguage sets

**Domain sets**. The metalanguage does not currently provide any shortcut syntax for getting domain sets, i.e. the sets corresponding to a type. The standard way to write a domain set is using the general `ConditionSet` notation. Here is the set for type `e`:

In [ ]:
%te Set x_e : True

In [ ]:
%te Set x_t : True

Sets of this form behave as you'd expect for simplification, despite their non-finiteness. 

In [ ]:
%te simplify (Set x_e: True) & {_c1, _c2}

In [ ]:
meta.meta.exec(te("(Set x_e: True) & {_c1, _c2}"))

In [ ]:
%te simplify (Set x_e: True) - (Set x_e: P(x))

Execution has caveats. While some special cases may work, the general case for set descriptions that could involve a non-finite domain set will not, and so it is often useful to only try to execute set-related expressions that have had a static simplify pass.

In [ ]:
meta.meta.exec(te("((Set x_e: True) & (Set x_e: False))"))

In [ ]:
# this will fail to compile, it needs static analysis to clean up the left case
with lamb.errors():
    meta.meta.exec(te("((Set x_e: True) & (Set x_e: True)) & {_c1, _c2}"))

In [ ]:
meta.meta.exec(te("((Set x_e: True) & (Set x_e: True)) & {_c1, _c2}").simplify_all())

**Complementation**. The metalanguage does not currently provide a unary complementation operator. However, this can be simulated using subtraction from the domain set, as in the above example, or as follows:

In [ ]:
%te simplify (Set x_t: True) - {True}

In [ ]:
# this set isn't finite, so can't simplify much
%te simplify (Set x_e: True) - {_c1, _c2}

In [ ]:
with tp("e").restrict_domain(count=4):
    x = %te simplify (Set x_e: True) - {_c1, _c2}
    display(x)

## Further python api notes on working with sets

The set api has various specialized pieces for dealing with set-related objects that it may be useful to know about.

### Conversions and tests on sets and containers

Several factory functions were mentioned earlier when discussion `ListedSet` construction. These are:

* `meta.sets.sset`: factory function for construction a pre-simplified `ListedSet`
* `meta.sets.emptyset`: factory function for empty sets specifically
* `meta.core.from_python_container`: given a python set/sequence container of `TypedExpr`s (including any container implementing `collections.abc.Set`), produce a corresponding `TypedExpr`.

Some useful tests for sets:

* `meta.sets.finite_safe`: returns true for certain finite set cases, namely those constructed from `ListedSet`s or `MetaTerm`s by set operations. These are cases where we are guaranteed finiteness.
* `meta.sets.is_emptyset`: for any of the ways of constructing a set, (syntactically) check if it's an empty set. For `ListedSet`s and `MetaTerm` sets, this is straightforward. For `ConditionSet`s, the semantic version of this is of course highly non-trivial, but this function returns true on condition sets with a simple `False` condition.
* `meta.core.is_concrete`: returns true on any `TypedExpr` data structure constructed (finitely) from `MetaTerm`s, including sets. These are essentially the cases where membership testing semantics is exactly that of naive set theory. (This doesn't accept sets constructed from operations.)

And finally, for conversion *out* of `TypedExpr` space:

* `ListedSet` and `MetaTerm` implement a member function `set()` that gives back a python `frozenset` of `TypedExpr`s. For a `MetaTerm`, this obviously would require a set type (and will raise if the term doesn't have one). If you want recursion, you probbly want one of the following two more general functions.
* `meta.core.to_python_container`: recursively converts `ListedSet` (and `Tuple`) structures, as well as their corresponding `MetaTerm` cases, into matching python `frozenset` (and `tuple`) objects.
* `meta.core.to_concrete`: similar to the previous, but accepts any `TypedExpr` and in a top-down way, converts as much as possible to python containers. This produces a useful normal form of sorts for working with the set/tuple container classes.

### Specialized member functions for set objects

**Characteristic functions**: Both `ConditionSet` and `ListedSet` implement a member function `to_characteristic`, which produces an `LFun` that checks for membership in the set.

In [ ]:
te("Set x_e : P_<e,t>(x)").to_characteristic()

In [ ]:
te("{_c1, _c2}").to_characteristic()

**Conversion**: See note on the `set()` member function above.

`ListedSet` can be converted to a `ConditionSet` via the member function `to_condition_set`, which builds a condition by disjoining equality checks on members.

In [ ]:
s = %te {_c1, _c2}
display(s, s.to_condition_set())

For obvious finiteness reasons, there is no general conversion in the other direction, but `ConditionSet` does support `eliminate` calls (like other `BindingOp`s) if the domain is finite, which generate a `ListedSet` (of `MetaTerm`s).

In [ ]:
s = %te Set x_{t} : True
display(s, s.eliminate())

(Careful with the domains, as `eliminate` calls will very happily attempt to produce *massive* sets if you let them! E.g. the above example tweaked to type `{{{t}}}` already blows up to a cardinality $2^{16}$ set. This wouldn't be a crazy size for a python object, but metalanguage objects at this size can get both unwieldy for humans and slow; you probably want to be working with compiled rather than simplified metalanguage code at this point.)